In [1]:
import pyximport; pyximport.install()
from utils import *
import time
import pandas as pd

In [2]:
from os import listdir
instances=listdir('Instances/')
instances=[x for x in [(x.split('.'))[0] for x in instances] if len(x) ==2]  

In [18]:
%%time
record ={}
start = time.time()
list_cost = []   
for e in instances:
    folder_instances = 'Instances/'
    folder_rpa_solutions = 'RPA_Solutions/'
    path = folder_instances+e+'.txt'
    path_solutions =folder_rpa_solutions+'Detailed_Solution_'+e+'.txt'

    count, number_of_customers, one, number_of_vehicles,\
    vect_cord, vect_domanda, vect_carico, vec_b, vec_l, capacity = parser_instance(path)  

    distance = distance_matrix(vect_cord)
    saving=saving_matrix(distance)
    
    dict_Saving_Order=savingOrder(saving)
    
    dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)
    
    list_vehicle, set_visited = init_list_of_vehicle_parallel(dict_l, vect_domanda, distance, number_of_vehicles)
    route_linehaul_parallel(number_of_customers, dict_l, list_vehicle, vect_domanda, set_visited, distance, capacity, number_of_vehicles) #funzione void
    route_backhaul_parallel(list_vehicle, dict_merge, dict_b, set_visited, vect_carico, distance, number_of_customers, capacity, number_of_vehicles) #funzione void
    tot_cost=sum([x.cost for x in list_vehicle])
    record[e]=round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2)
    list_cost.append([e,tot_cost])
duration = time.time()-start

CPU times: user 2min 3s, sys: 374 ms, total: 2min 3s
Wall time: 2min 3s


In [19]:
df_cost = pd.DataFrame(list_cost, columns = ['Instances', 'Best_Solution']) 

In [21]:
df_cost.to_excel('Risultati/costo_parallelSolution.xls')

In [4]:
tot_errore=0
for e in record.keys():
    tot_errore += record[e]

tot_errore/len(record)

88.42000000000003

In [5]:
df_data = pd.DataFrame({'': [number_of_customers, capacity, number_of_vehicles,
                            len(set_visited)-1, tot_cost, extract_total_cost_BP(path_solutions),
                            round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2), duration]},
                       index=['Customers', 'Max Load', 'Routes Of the Solution', 'nodes visited',
                              'Total Cost', 'Total Cost BP', 'Error', 'Time for Instance'
                             ])

In [7]:
df_data

,
Customers,38.000000
Max Load,2750.000000
Routes Of the Solution,7.000000
nodes visited,38.000000
Total Cost,441394.047666
Total Cost BP,239479.000000
Error,84.310000
Time for Instance,0.048302


In [ ]:
#support
count = 0
best_route = []
flag = False

with open(path_solutions) as file:
    for line in file:
        if "ROUTE" in line:
            flag = True
        if flag:
            count+=1
        if count == 7:
            best_route.append([int(x) for x in line.split()])
            flag=False
            count=0


In [ ]:
best_route

In [ ]:
my_route=[x.route for x in list_vehicle]
my_route

In [ ]:
best_cost=0
for element in best_route:
    for i,v in enumerate(element):
        if i > 0:
            best_cost+=distance[element[i-1]][element[i]]

In [ ]:
my_cost=0
for element in my_route:
    for i,v in enumerate(element):
        if i > 0:
            my_cost+=distance[element[i-1]][element[i]]

In [ ]:
best_cost

In [ ]:
my_cost

In [ ]:
round((my_cost * 100) / best_cost -100 , 2)

In [ ]:
record

In [8]:
list_record=[]
for e in record.keys():
    list_record.append([e,record[e]])

In [9]:
df = pd.DataFrame(list_record, columns = ['Instances', 'Error']) 

In [14]:
df2 = pd.DataFrame([['Total Average', tot_errore/len(record)]], columns = ['Instances', 'Error']) 

In [16]:
df.append(df2)

,Instances,Error
0,G2,131.25
1,K1,94.74
2,J4,88.53
3,G3,133.62
4,G1,93.87
...,...,...
57,I5,63.96
58,H1,97.72
59,I4,64.42
60,D3,84.31


In [17]:
df.to_csv('Risultati/parallelo_completo.csv')